# Applying Machine Learning to Sentiment Analysis

The topics that we will cover include the following:
    - Cleaning and processing text data
    - Building feature vector from text documents
    - Trainng a machine learning model to classify positive and negative movie reviews

## Obtaining the movie review dataset

We can download a compressed archive of the movie review dataset(80.2 MB) from
[https://ai.stanford.edu/~amaas/data/sentiment/] as Gzip-compressed tarball archive.

To Extract data from Gzip archive you can use 7-zip application. You may need to istall it from [http://www.7-zip.org]

Alternatively, you can directly unpack the Gzip-compressed tarball archive directly in Python as follows:

```
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()
```

In [1]:
# Load labraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import shap # labirary for SHAP analysis

%matplotlib inline

# Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

In [ ]:
import tarfile

target = 'aclImdb_v1.tar.gz'

with tarfile.open(target, 'r:gz') as tar:
    tar.extractall()

### Preprocessing the movie dataset into more convenient format

In [ ]:
! pip install pyprind

In [ ]:
import pyprind
import pandas as pd
import os

# change the `basepath` to the directory of the
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']
df.head()

In [ ]:
import numpy as np

# Shuffling the DataFrame:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.head()

In [ ]:
# Optional: Saving the assembled data as CSV file:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [2]:
# Read from file into DataFrame:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [3]:
# Data Frame Shape:
df.shape

(50000, 2)

In [4]:
# New Count HER2 Positive and Negative after resolving Unknown problem
df['sentiment'].value_counts()

0    25000
1    25000
Name: sentiment, dtype: int64

## Introducing the bag-of-words model

### Transforming documents into feature vectors

By calling the fit_transform method on CountVectorizer, we just constructed the vocabulary of the bag-of-words model and transformed the following three sentences into sparse feature vectors:

1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

Now let us print the contents of the vocabulary to get a better understanding of the underlying concepts:

In [6]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [7]:
for k,v in sorted(count.vocabulary_.items()):
    print(k, v)

and 0
is 1
one 2
shining 3
sun 4
sweet 5
the 6
two 7
weather 8


As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary, which maps the unique words that are mapped to integer indices. Next let us print the feature vectors that we just created:

Each index position in the feature vectors shown here corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the rst feature at index position 0 resembles the count of the word and, which only occurs in the last document, and the word is at index position 1 (the 2nd feature in the document vectors) occurs in all three sentences. Those values in the feature vectors are also called the raw term frequencies: tf *(t,d)* — the number of times a term t occurs in a document *d*.

In [8]:
# 'The sun is shining',
# Vocab   Val  VocabPosition
# and      0      1
# is       1      2
# one      0      3
# shining  1      4
# sun      1      5
# sweet    0      6
# the      1      7
# two      0      8
# weather  0      9

# 'The weather is sweet',
# 'The sun is shining, the weather is sweet, and one and one is two'



In [9]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### Assessing word relevancy via term frequency-inverse document frequency

When we are analyzing text data, we often encounter words that occur across multiple documents from both classes. Those frequently occurring words typically don't contain useful or discriminatory information. In this subsection, we will learn about a useful technique called term frequency-inverse document frequency (tf-idf) that can be used to downweight those frequently occurring words in the feature vectors. The tf-idf can be de considered as the product of the term frequency and the inverse document frequency:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$
Here the tf(t, d) is the term frequency that we introduced in the previous section, and the inverse document frequency idf(t, d) can be calculated as:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$
where $n_d$ is the total number of documents, and df(d, t) is the number of documents d that contain the term t. Note that adding the constant 1 to the denominator is optional and serves the purpose of assigning a non-zero value to terms that occur in all training samples; the log is used to ensure that low document frequencies are not given too much weight.

Scikit-learn implements yet another transformer, the TfidfTransformer, that takes the raw term frequencies from CountVectorizer as input and transforms them into tf-idfs:

In [10]:
# Set precision
np.set_printoptions(precision=2)

# Import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer

# instanciate TfidfTransformer object
tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
# apply TfidfTransformer to our list of documents
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


As we saw in the previous subsection, the word **is** had the largest term frequency in the 3rd document, being the most frequently occurring word. However, after transforming the same feature vector into *tf-idfs*, we see that the word is is now associated with a relatively small tf-idf (0.45) in document 3 since it is also contained in documents 1 and 2 and thus is unlikely to contain any useful, discriminatory information.

However, if we'd manually calculated the tf-idfs of the individual terms in our feature vectors, we'd have noticed that the TfidfTransformer calculates the tf-idfs slightly differently compared to the standard textbook equations that we saw earlier. The equations for the idf and tf-idf that were implemented in scikit-learn are:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$
The tf-idf equation that was implemented in scikit-learn is as follows:

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$
While it is also more typical to normalize the raw term frequencies before calculating the tf-idfs, the TfidfTransformer normalizes the tf-idfs directly.

By default (norm='l2'), scikit-learn's TfidfTransformer applies the L2-normalization, which returns a vector of length 1 by dividing an un-normalized feature vector v by its L2-norm:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$
To make sure that we understand how TfidfTransformer works, let us walk through an example and calculate the tf-idf of the word is in the 3rd document.

The word **is** has a term frequency of 3 (tf = 3) in document 3, and the document frequency of this term is 3 since the term is occurs in all three documents (df = 3). Thus, we can calculate the idf as follows:

$$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$$
Now in order to calculate the tf-idf, we simply need to add 1 to the inverse document frequency and multiply it by the term frequency:

$$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$$

In [11]:
# Example of calcualtion TF-IDF for word "is" using Python implementation
tf_is = 3     # TF- term frequency; number of times that "is" in the 3rd sentance
n_docs = 3    # number of documents that "is" apears in 
idf_is = np.log((n_docs+1) / (3+1))        # calculationg IDF for "is" 
tfidf_is = tf_is * (idf_is + 1)            # claculating TF-IDF for "is"         
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


If we repeated these calculations for all terms in the 3rd document, we'd obtain the following tf-idf vectors: [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]. However, we notice that the values in this feature vector are different from the values that we obtained from the TfidfTransformer that we used previously. The nal step that we are missing in this tf-idf calculation is the L2-normalization, which can be applied as follows:


$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$


As we can see, the results match the results returned by scikit-learn's TfidfTransformer (below). Since we now understand how tf-idfs are calculated, let us proceed to the next sections and apply those concepts to the movie review dataset.

In [12]:
# Calculate TF_IDF for 3rd sentance
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf

array([3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29])

In [13]:
# Apply L2 norm - normilized TF-IDF vector 
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

## Cleaning text data

In [14]:
# Look at last 50 characters of the first review
# we would see HTML notations there
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [15]:
import re

# Create preprocessor function to eliminate HTML tags and other characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [16]:
# Applying preprocessor function
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [17]:
# Processing emoji - example
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [18]:
# Look at the entire first review before processing
df.iloc[0, 0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [19]:
# apply preprocessing to our movie reviews
df['review'] = df['review'].apply(preprocessor)

# See the first review after preprocessing
df.iloc[0, 0]

'in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover the mur

In [20]:
df.iloc[1, 0]

'ok so i really like kris kristofferson and his usual easy going delivery of lines in his movies age has helped him with his soft spoken low energy style and he will steal a scene effortlessly but disappearance is his misstep holy moly this was a bad movie i must give kudos to the cinematography and and the actors including kris for trying their darndest to make sense from this goofy confusing story none of it made sense and kris probably didn t understand it either and he was just going through the motions hoping someone would come up to him and tell him what it was all about i don t care that everyone on this movie was doing out of love for the project or some such nonsense i ve seen low budget movies that had a plot for goodness sake this had none zilcho nada zippo empty of reason a complete waste of good talent scenery and celluloid i rented this piece of garbage for a buck and i want my money back i want my 2 hours back i invested on this grade f waste of my time don t watch this 

## Processing documents into tokens

In [21]:
# Import Library for Stemmering 
# Stemmers remove morphological affixes from words, leaving only the word stem.
from nltk.stem.porter import PorterStemmer

# get PorterStemmer() object
porter = PorterStemmer()

# tokenization your text - split sentance into seperate words
def tokenizer(text):
    return text.split()

# Tokenizing your words
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()
#data[‘pre_process’]=data[‘pre_process’].apply(lambda x: “ “.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))

In [22]:
# Tokenizing example
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [23]:
# Tokenizing and Stemming example
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [24]:
# Doqnload stop-words
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\4471082\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.corpus import stopwords

# Get the list of stop words
stop = stopwords.words('english')

# Example of removing stop words from the list 
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

#from nltk.corpus import stopwords
#stop = stopwords.words(‘english’)
#data[‘pre_process’]=data[‘pre_process’].apply(lambda x: “ “.join([x for x in x.split() if x not in stop]))

['runner', 'like', 'run', 'run', 'lot']

In [26]:
# Example of English stop-words
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Training a logistic regression model for document classification

Strip HTML and punctuation to speed up the GridSearch later:

In [27]:
# Split our review dataset into train and test sets
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [29]:
print("TFIDF Vectorizer……")
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

TFIDF Vectorizer……


In [31]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000)

In [33]:
clf.fit(tf_x_train,y_train)

LogisticRegression(max_iter=1000)

In [34]:
y_test_pred=clf.predict(tf_x_test)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89     12527
           1       0.89      0.90      0.89     12473

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



In [43]:
from sklearn.model_selection import GridSearchCV

# defining parameter range 
param_grid = {'C': [1.0, 5.0, 10.0, 50.0, 100.0]}

grid = GridSearchCV(LogisticRegression(max_iter=1000),
                    scoring='accuracy',
                    cv=5,
                    param_grid = param_grid,
                    refit = True,
                    verbose = 3,
                    n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(tf_x_train, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 

grid_predictions = grid.predict(tf_x_test) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 
             

Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'C': 10.0}
              precision    recall  f1-score   support

           0       0.90      0.89      0.90     12527
           1       0.89      0.90      0.90     12473

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000



In [29]:
# Import library for Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

# Parameters for GridSearch
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
# Creating Pipeline with Logistic Regression Model 
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

# Search for bet Logisting Regression Model
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

**Important Note about n_jobs**

Please note that it is highly recommended to use `n_jobs=-1` (instead of `n_jobs=1`) in the previous code example to utilize all available cores on your machine and speed up the grid search. However, some Windows users reported issues when running the previous code with the `n_jobs=-1` setting related to pickling the tokenizer and tokenizer_porter functions for multiprocessing on Windows. Another workaround would be to replace those two functions, `[tokenizer, tokenizer_porter]`, with `[str.split]`. However, note that the replacement by the simple str.split would not support stemming.

**Important Note about the running time**

Executing the following code cell **may take up to 30-60 min** depending on your machine, since based on the parameter grid we defined, there are 22235 + 22235 = 240 models to fit.

If you do not wish to wait so long, you could reduce the size of the dataset by decreasing the number of training samples, for example, as follows:

`
X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values
`

However, note that decreasing the training set size to such a small number will likely result in poorly performing models. Alternatively, you can delete parameters from the grid above to reduce the number of models to fit -- for example, by using the following:

`
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0]},
              ]
`

In [30]:
# Train Model 
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


One or more of the test scores are non-finite: [ nan  nan  nan  nan 0.89 0.89 0.89 0.89  nan  nan  nan  nan 0.9  0.89
 0.9  0.89  nan  nan  nan  nan 0.89 0.88 0.89 0.88  nan  nan  nan  nan
 0.88 0.87 0.88 0.88  nan  nan  nan  nan 0.87 0.86 0.88 0.87  nan  nan
  nan  nan 0.87 0.86 0.88 0.87]
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd",...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
           

In [31]:
# Print best model parameters
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

# Print Cross-Validation accuracy 
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000002864D498310>} 
CV Accuracy: 0.897


In [32]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.899


In [40]:
gs_lr_tfidf.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<function tokenizer at 0x000002864D498310>)),
                ('clf', LogisticRegression(C=10.0, random_state=0))])

In [41]:
dfn = df.copy()
dfn.head()


,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about w...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


In [47]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

# Example of removing stop words from the list 
dfn['review_t'] = dfn['review'].apply(lambda x: tokenizer_porter(x) )
#dfn_tokenizer = [w for w in tokenizer_porter(dfn['review']) if w not in stop]

In [55]:
dfn['review_t1'] = dfn['review_t'].apply(lambda x: w for w in x if w not in stop)

NameError: name 'x' is not defined

In [57]:
dfn['review_t2'] = dfn['review_t1'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [59]:
dfn['review_t2'][0]

'1974 teenag martha moxley maggi grace move high class area bell greenwich connecticut mischief night eve halloween wa murder backyard hous murder remain unsolv twenti two year later writer mark fuhrman christoph meloni former la detect ha fallen disgrac perjuri j simpson trial move idaho decid investig case hi partner stephen week andrew mitchel purpos write book local squirm welcom support retir detect steve carrol robert forster wa charg investig 70 discov crimin net power money cover murder murder greenwich good tv movi true stori murder fifteen year old girl wa commit wealthi teenag whose mother wa kennedi power rich famili use influenc cover murder twenti year howev snoopi detect convict perjur disgrac wa abl disclos hideou crime wa commit screenplay show investig mark last day martha parallel lack emot dramat vote seven titl brazil avail'

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(dfn['review_t2'])

In [63]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    text_tf, dfn['sentiment'], test_size=0.50, random_state=12)

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(C=10.0, penalty='l2', max_iter=1000, n_jobs=-1, random_state=12)

lr.fit(X_train1, y_train1)
predicted= lr.predict(X_test1)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test1, predicted))

Logistic Regression Accuracy: 0.88944


## Serializing fitted scikit-learn estimators

After we trained the logistic regression model as shown above, we know save the classifier along woth the stop words, Porter Stemmer, and HashingVectorizer as serialized objects to our local disk so that we can use the fitted classifier in our web application later.

In [49]:
import pickle
#import dill as pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

#pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
#pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)
#pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'))   
#pickle.dump(grid, open(os.path.join(dest, 'classifier.pkl'), 'wb'))
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(grid, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

Next, we save the HashingVectorizer as in a separate file so that we can import it later. Save it to `movieclassifier` folder.

In [ ]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

`Overwriting movieclassifier/vectorizer.py`

After executing the serialization code and creating `vectorizer.py` file in the `movieclassifier` folder, we can create new IPython notebook run from `movieclassifier` folder to check if the objects were serialized correctly.
When you run new IPython notebook from `movieclassifier` folder then run the following code:

In [ ]:
import os

path = os.getcwd()

print(path)

In [ ]:
import os
os.chdir('movieclassifier')

In [ ]:
path = os.getcwd()

print(path)

In [50]:
#import pickle
import dill as pickle
import re
import os
#from vectorizer import vect
from sklearn.feature_extraction.text import HashingVectorizer

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

stop = pickle.load(open(os.path.join('movieclassifier\pkl_objects', 'stopwords.pkl'), 'rb'))

# Loading model to compare the results
model = pickle.load(open(os.path.join('movieclassifier\pkl_objects', 'classifier.pkl'), 'rb'))

## Setting up a SQLite database for data storage


Before you execute this code, please make sure that you are currently in the movieclassifier directory.

Note that we are still on the "movieclassifier" subdirectory:

In [2]:
import os
os.getcwd()

'C:\\Users\\4471082\\Documents\\My Project\\Movieclassifier'

In [ ]:
import sqlite3
import os

if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [ ]:
print(results)